In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 27.1 MB/s 
     |████████████████████████████████| 788 kB 58.3 MB/s 
     |████████████████████████████████| 182 kB 58.7 MB/s 
     |████████████████████████████████| 5.8 MB 55.4 MB/s 
     |████████████████████████████████| 981 kB 63.4 MB/s 
     |████████████████████████████████| 1.6 MB 58.2 MB/s 
     |████████████████████████████████| 19.7 MB 123.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 200 kB 67.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 7.6 MB 49.1 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=daa0f4980da474741f7ce353247cc23e08086d199a6509b4871c0961b0426436
  Stored in directory: /root/.cache/pip/wheels/3d/9f/9d/d806a

In [7]:
import flair

In [8]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus, TREC_6
from flair.models import TARSTagger
from flair.trainers import ModelTrainer

# 1. define label names in natural language since some datasets come with cryptic set of labels
label_name_map = {'Organisation': 'Organisation', 
                    'Location': 'Location',
                    'Person': 'Person',
                    'Temporal': 'Temporal',
                    'Quantity': 'Quantity',
                    'Weapon': 'Weapon',
                    'DocumentReference': 'Document Reference',
                    'MilitaryPlatform': 'Military Platform',
                    'Nationality': 'Nationality',
                    'Money': 'Money'
                  }

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/My Drive/splits'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='val.conll')

corpus = corpus.downsample(0.05)

# 3. what label do you want to predict?
label_type = 'ner'

# 4. make a label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

# 5. start from our existing TARS base model for English
tars = TARSTagger.load("tars-ner")

# 5a: alternatively, comment out previous line and comment in next line to train a new TARS model from scratch instead
# tars = TARSClassifier(embeddings="bert-base-uncased")

# 6. switch to a new task (TARS can do multiple tasks so you must define one)
tars.add_and_switch_to_new_task(task_name="ner",
                                label_dictionary=label_dict,
                                label_type=label_type,
                                )

# 7. initialize the text classifier trainer
trainer = ModelTrainer(tars, corpus)

# 8. start the training
trainer.train(base_path='resources/taggers/trec',  # path to store the model artifacts
              learning_rate=0.02,  # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4,  # optionally set this if transformer is too much for your machine
              max_epochs=20,  # terminate after 10 epochs
              )

2022-12-14 13:15:17,019 Reading data from /content/drive/My Drive/splits
2022-12-14 13:15:17,022 Train: /content/drive/My Drive/splits/train.conll
2022-12-14 13:15:17,025 Dev: /content/drive/My Drive/splits/val.conll
2022-12-14 13:15:17,028 Test: /content/drive/My Drive/splits/test.conll
2022-12-14 13:15:19,734 Computing label dictionary. Progress:


39it [00:00, 2277.42it/s]

2022-12-14 13:15:19,801 Dictionary created for label 'ner' with 9 values: Organisation (seen 43 times), Location (seen 27 times), Person (seen 19 times), Temporal (seen 12 times), Quantity (seen 8 times), MilitaryPlatform (seen 4 times), DocumentReference (seen 3 times), Weapon (seen 2 times)


2022-12-14 13:15:20,911 https://nlp.informatik.hu-berlin.de/resources/models/tars-ner/tars-ner.pt not found in cache, downloading to /tmp/tmp6wtmv3ai


100%|██████████| 1421680237/1421680237 [02:10<00:00, 10893762.93B/s]

...

2022-12-14 14:50:03,703 Evaluating as a multi-label problem: False
2022-12-14 14:50:03,717 DEV : loss 0.2618752121925354 - f1-score (micro avg)  0.5385
2022-12-14 14:50:03,720 Epoch    20: reducing learning rate of group 0 to 1.2500e-03.
2022-12-14 14:50:03,725 BAD EPOCHS (no improvement): 4


2022-12-14 14:50:09,985 ----------------------------------------------------------------------------------------------------
2022-12-14 14:50:09,991 loading file resources/taggers/trec/best-model.pt
2022-12-14 14:50:30,702 SequenceTagger predicts: Dictionary with 5 tags: O, S-entity, B-entity, E-entity, I-entity


100%|██████████| 5/5 [00:22<00:00,  4.41s/it]

2022-12-14 14:50:53,725 Evaluating as a multi-label problem: False
2022-12-14 14:50:53,751 0.7222	0.5652	0.6341	0.4643
2022-12-14 14:50:53,753 
Results:
- F-score (micro) 0.6341
- F-score (macro) 0.6106
- Accuracy 0.4643

By class:
              precision    recall  f1-score   support

Organisation     1.0000    0.6364    0.7778        11
    Location     0.5000    0.7500    0.6000         4
    Quantity     0.3333    0.2500    0.2857         4
      Person     1.0000    1.0000    1.0000         1
      Weapon     0.0000    0.0000    0.0000         2
    Temporal     1.0000    1.0000    1.0000         1

   micro avg     0.7222    0.5652    0.6341        23
   macro avg     0.6389    0.6061    0.6106        23
weighted avg     0.7101    0.5652    0.6130        23

2022-12-14 14:50:53,756 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6341463414634146,
 'dev_score_history': [0.5517241379310345,
  0.5925925925925927,
  0.5999999999999999,
  0.6206896551724138,
  0.4999999999999999,
  0.6153846153846154,
  0.4444444444444445,
  0.4999999999999999,
  0.5217391304347826,
  0.5217391304347826,
  0.4827586206896552,
  0.5384615384615384,
  0.5185185185185186,
  0.3636363636363637,
  0.4,
  0.38461538461538464,
  0.4444444444444445,
  0.5,
  0.4999999999999999,
  0.5384615384615384],
 'train_loss_history': [0.2823573362654988,
  0.20272142808693466,
  0.16957946423061993,
  0.1539023671229054,
  0.14630435792255234,
  0.12762798333025574,
  0.1129232270373009,
  0.09700362274099543,
  0.057954863769876165,
  0.055079912683708404,
  0.07126743210353265,
  0.06385656289153296,
  0.03745323821451484,
  0.03147965378548677,
  0.034848284023485485,
  0.03216016756636096,
  0.027938329132153487,
  0.028939166094265428,
  0.030591555835231637,
  0.01983435118753096],
 'dev_loss_history': [0.14699797332286835,
  0

In [4]:
Out[2]

KeyError: ignored